In [1]:
import torch.nn            as nn
import numpy               as np
import matplotlib.pyplot   as plt
import torch.nn.functional as F
import networkx            as nx
import itertools
import torch

from os                   import path
from torch_geometric.data import Data
from pymatgen.core        import Structure
from torch.nn             import Linear

import sys
sys.path.append('../')
import MP.MP_library as MPL

# Graph embedding

In [2]:
distance_threshold = 5
y = 1
cell, composition, concentration, positions = MPL.information_from_VASPfile('.',
                                                                            'POSCAR')

nodes, edges, attributes = MPL.graph_POSCAR_encoding(None,
                                                    cell,
                                                    composition,
                                                    concentration,
                                                    positions,
                                                    distance_threshold=distance_threshold
                                                   )

#attributes = attributes * 100
temp = Data(x=nodes,
            edge_index=edges.t().contiguous(),
            edge_attr=attributes.flatten(),
            y=torch.tensor([y], dtype=torch.float)
           )
temp

Data(x=[71, 4], edge_index=[2, 586], edge_attr=[586], y=[1])

In [3]:
def allocate_atom_4n(P1, P2, P3, P4, r1, r2, r3, r4):
    """
    Trilateration function to find the coordinates of the unknown point given three reference points
    and their corresponding distances to the unknown point.

    Args:
        P1 (numpy.ndarray): Coordinates of the first reference point.
        P2 (numpy.ndarray): Coordinates of the second reference point.
        P3 (numpy.ndarray): Coordinates of the third reference point.
        r1 (float): Distance from the unknown point to the first reference point.
        r2 (float): Distance from the unknown point to the second reference point.
        r3 (float): Distance from the unknown point to the third reference point.

    Returns:
        Tuple[numpy.ndarray, numpy.ndarray]: Coordinates of the unknown point in two possible solutions.
    """

    # Calculate vector representations of the reference points
    p1 = np.array([0, 0, 0])
    p2 = np.array([P2[0] - P1[0], P2[1] - P1[1], P2[2] - P1[2]])
    p3 = np.array([P3[0] - P1[0], P3[1] - P1[1], P3[2] - P1[2]])

    # Calculate vectors representing the directions from the first reference point
    # to the second and third reference points
    v1 = p2 - p1
    v2 = p3 - p1

    # Calculate unit vectors in the Xn, Yn, and Zn directions
    Xn = v1 / np.linalg.norm(v1)
    tmp = np.cross(v1, v2)
    Zn = tmp / np.linalg.norm(tmp)
    Yn = np.cross(Xn, Zn)

    # Calculate dot products and distances
    i = np.dot(Xn, v2)
    d = np.dot(Xn, v1)
    j = np.dot(Yn, v2)

    # Calculate coordinates of the unknown point in two possible solutions
    X = (np.power(r1, 2) - np.power(r2, 2) + np.power(d, 2)) / (2 * d)
    Y = (np.power(r1, 2) - np.power(r3, 2) + np.power(i, 2) + np.power(j, 2)) / (2 * j) - X * i / j
    Z1_square = np.power(r1, 2) - np.power(X, 2) - np.power(Y, 2)
    
    if Z1_square < -1e-4:
        return None
    
    Z1 = np.sqrt(max(0, Z1_square))
    Z2 = -Z1

    # Calculate the coordinates of the unknown point for both solutions
    K1 = P1 + X * Xn + Y * Yn + Z1 * Zn
    K2 = P1 + X * Xn + Y * Yn + Z2 * Zn
    
    if (np.abs(np.linalg.norm(K1 - P4) - r4) < 1e-2) or (np.abs(np.linalg.norm(K2 - P4) - r4) < 1e-2):
        if np.abs(np.linalg.norm(K1 - P4) - r4) < np.abs(np.linalg.norm(K2 - P4) - r4):
            return K1
        return K2
    return None

In [4]:
def get_n_connected(idx_0, cartesian_positions, edge_indexes, edge_attributes):
    n_connected = 0
    idx_connected = []
    for idx_t in list(cartesian_positions.keys()):
        if get_distance_attribute(idx_0, idx_t, edge_indexes, edge_attributes) is not None:
            n_connected += 1
            idx_connected.append(idx_t)
    
    return n_connected, idx_connected


def allocate_atom_n(P1, P2, P3, r1, r2, r3):
    """
    Trilateration function to find the coordinates of the unknown point given three reference points
    and their corresponding distances to the unknown point.

    Args:
        P1 (numpy.ndarray): Coordinates of the first reference point.
        P2 (numpy.ndarray): Coordinates of the second reference point.
        P3 (numpy.ndarray): Coordinates of the third reference point.
        r1 (float): Distance from the unknown point to the first reference point.
        r2 (float): Distance from the unknown point to the second reference point.
        r3 (float): Distance from the unknown point to the third reference point.

    Returns:
        Tuple[numpy.ndarray, numpy.ndarray]: Coordinates of the unknown point in two possible solutions.
    """

    # Calculate vector representations of the reference points
    p1 = np.array([0, 0, 0])
    p2 = np.array([P2[0] - P1[0], P2[1] - P1[1], P2[2] - P1[2]])
    p3 = np.array([P3[0] - P1[0], P3[1] - P1[1], P3[2] - P1[2]])

    # Calculate vectors representing the directions from the first reference point
    # to the second and third reference points
    v1 = p2 - p1
    v2 = p3 - p1

    # Calculate unit vectors in the Xn, Yn, and Zn directions
    Xn = v1 / np.linalg.norm(v1)
    tmp = np.cross(v1, v2)
    Zn = tmp / np.linalg.norm(tmp)
    Yn = np.cross(Xn, Zn)

    # Calculate dot products and distances
    i = np.dot(Xn, v2)
    d = np.dot(Xn, v1)
    j = np.dot(Yn, v2)

    # Calculate coordinates of the unknown point in two possible solutions
    X = (np.power(r1, 2) - np.power(r2, 2) + np.power(d, 2)) / (2 * d)
    Y = (np.power(r1, 2) - np.power(r3, 2) + np.power(i, 2) + np.power(j, 2)) / (2 * j) - X * i / j
    Z1_square = np.power(r1, 2) - np.power(X, 2) - np.power(Y, 2)
    
    if Z1_square < -1e-4:
        return None
    
    Z1 = np.sqrt(max(0, Z1_square))
    Z2 = -Z1

    # Calculate the coordinates of the unknown point for both solutions
    K1 = P1 + X * Xn + Y * Yn + Z1 * Zn
    K2 = P1 + X * Xn + Y * Yn + Z2 * Zn
    return K1


def get_distance_attribute(index0, index1, edge_indexes, edge_attributes):
    """Get the distance attribute between two nodes with given indices.

    Args:
        index0          (int):        Index of the first node.
        index1          (int):        Index of the second node.
        edge_indexes    (np.ndarray): Array containing indices of connected nodes for each edge.
        edge_attributes (np.ndarray): Array containing attributes corresponding to each edge.

    Returns:
        float: The distance attribute between the two nodes.
        False if index0, index1 not linked.
    """
    
    # Create a mask to find matching edges
    mask_direct  = (edge_indexes[0] == index0) & (edge_indexes[1] == index1)
    mask_reverse = (edge_indexes[0] == index1) & (edge_indexes[1] == index0)
    
    # Check if any edge satisfies the conditions
    matching_edge_indices = np.where(mask_direct | mask_reverse)[0]
    
    if len(matching_edge_indices) == 0:
        return None  # The pair is not linked
    
    # Get the distance attribute from the first matching edge
    distance_attribute = edge_attributes[matching_edge_indices[0]]
    
    return distance_attribute


def find_initial_basis(total_particles, edge_indexes, edge_attributes):
    for idx_0 in range(total_particles):
        for idx_1 in np.arange(idx_0+1, total_particles):
            for idx_2 in np.arange(idx_1+1, total_particles):
                condition_01 = (get_distance_attribute(idx_0, idx_1, edge_indexes, edge_attributes) is not None)
                condition_12 = (get_distance_attribute(idx_1, idx_2, edge_indexes, edge_attributes) is not None)
                condition_02 = (get_distance_attribute(idx_0, idx_2, edge_indexes, edge_attributes) is not None)
                if condition_01 and condition_12 and condition_02:
                    return idx_0, idx_1, idx_2


def find_valid_reference(n_connected, idx_connected, edge_indexes, edge_attributes):
    if n_connected >= 4:
        for i in range(n_connected):
            for j in np.arange(i+1, n_connected):
                for k in np.arange(j+1, n_connected):
                    for l in np.arange(k+1, n_connected):
                        idx_0 = idx_connected[i]
                        idx_1 = idx_connected[j]
                        idx_2 = idx_connected[k]
                        idx_3 = idx_connected[l]

                        r0 = cartesian_positions[idx_0]
                        r1 = cartesian_positions[idx_1]
                        r2 = cartesian_positions[idx_2]
                        r3 = cartesian_positions[idx_3]

                        # Get necessary distances
                        d_0n = get_distance_attribute(idx_0, idx, edge_indexes, edge_attributes)
                        d_1n = get_distance_attribute(idx_1, idx, edge_indexes, edge_attributes)
                        d_2n = get_distance_attribute(idx_2, idx, edge_indexes, edge_attributes)
                        d_3n = get_distance_attribute(idx_3, idx, edge_indexes, edge_attributes)

                        temp_position = allocate_atom_4n(r0, r1, r2, r3, d_0n, d_1n, d_2n, d_3n)

                        if temp_position is not None:
                            return temp_position
    else:
        for i in range(n_connected):
            for j in np.arange(i+1, n_connected):
                for k in np.arange(j+1, n_connected):
                    idx_0 = idx_connected[i]
                    idx_1 = idx_connected[j]
                    idx_2 = idx_connected[k]

                    r0 = cartesian_positions[idx_0]
                    r1 = cartesian_positions[idx_1]
                    r2 = cartesian_positions[idx_2]

                    # Get necessary distances
                    d_0n = get_distance_attribute(idx_0, idx, edge_indexes, edge_attributes)
                    d_1n = get_distance_attribute(idx_1, idx, edge_indexes, edge_attributes)
                    d_2n = get_distance_attribute(idx_2, idx, edge_indexes, edge_attributes)

                    temp_position = allocate_atom_n(r0, r1, r2, d_0n, d_1n, d_2n)

                    if temp_position is not None:
                        return temp_position

In [5]:
r0 = np.array([0.08650538, 0.18660925, 0.37290699])
r1 = np.array([0.08726176, 0.68821608, 0.37662460])
r2 = np.array([0.41807971, 0.19089353, 0.38322752])
r3 = np.array([0.75213915, 0.18744935, 0.37758075])
d  = np.array([0.42024979, 0.68779296, 0.37657149]) 

# Distances
d_0n = np.linalg.norm([r0 - d])
d_1n = np.linalg.norm([r1 - d])
d_2n = np.linalg.norm([r2 - d])
d_3n = np.linalg.norm([r3 - d])

#allocate_atom_n(r0, r1, r2, r3, d_0n, d_1n, d_2n, d_3n)
allocate_atom_n(r0, r1, r2, d_0n, d_1n, d_2n)

array([0.42024979, 0.68779296, 0.37657149])

In [6]:
# Extract indexes and attributes from the graph
edge_indexes    = edges.T.detach().cpu().numpy().copy()
edge_attributes = attributes.detach().cpu().numpy().copy().flatten()

# Define the number of atoms in the graph
total_particles = len(nodes)

# Select three initial particles which are interconnected
idx_0, idx_1, idx_2 = find_initial_basis(total_particles, edge_indexes, edge_attributes)

# Get necessary distances
d_01 = get_distance_attribute(idx_0, idx_1, edge_indexes, edge_attributes)
d_02 = get_distance_attribute(idx_0, idx_2, edge_indexes, edge_attributes)
d_12 = get_distance_attribute(idx_1, idx_2, edge_indexes, edge_attributes)

# Reference the first three atoms
x2 = (np.power(d_01, 2) + np.power(d_02, 2) - np.power(d_12, 2)) / (2 * d_01)
y2 = np.sqrt(np.power(d_02, 2) - np.power(x2, 2))

# Impose three particles at the beginning
cartesian_positions = {
    idx_0: np.array([0,    0,  0]),
    idx_1: np.array([d_01, 0,  0]),
    idx_2: np.array([x2,   y2, 0])
}

print(np.linalg.norm(cartesian_positions[idx_1] - cartesian_positions[idx_0]), d_01)
print(np.linalg.norm(cartesian_positions[idx_2] - cartesian_positions[idx_0]), d_02)
print(np.linalg.norm(cartesian_positions[idx_1] - cartesian_positions[idx_2]), d_12)

4.183528900146484 4.183529
4.217043399810791 4.2170434
4.213690280914307 4.2136903


In [7]:
# Extract indexes and attributes from the graph
edge_indexes    = edges.T.detach().cpu().numpy().copy()
edge_attributes = attributes.detach().cpu().numpy().copy().flatten()

# Define the number of atoms in the graph
total_particles = len(nodes)

# Select three initial particles which are interconnected
idx_0, idx_1, idx_2 = find_initial_basis(total_particles, edge_indexes, edge_attributes)

# Get necessary distances
d_01 = get_distance_attribute(idx_0, idx_1, edge_indexes, edge_attributes)
d_02 = get_distance_attribute(idx_0, idx_2, edge_indexes, edge_attributes)
d_12 = get_distance_attribute(idx_1, idx_2, edge_indexes, edge_attributes)

# Reference the first three atoms
x2 = (np.power(d_01, 2) + np.power(d_02, 2) - np.power(d_12, 2)) / (2 * d_01)
y2 = np.sqrt(np.power(d_02, 2) - np.power(x2, 2))

# Impose three particles at the beginning
cartesian_positions = {
    idx_0: np.array([0,    0,  0]),
    idx_1: np.array([d_01, 0,  0]),
    idx_2: np.array([x2,   y2, 0])
}

all_idxs = np.delete(np.arange(total_particles),
                     [idx_0, idx_1, idx_2])

# Initialized to 3 for three connections
n_connected_list = all_idxs.tolist()
idx_0, idx_1, idx_2

(0, 2, 4)

In [8]:
while len(n_connected_list):  # Goes until all particles have been studied
    # Using a first-one-first-out approach
    idx = n_connected_list[0]

    n_connected, idx_connected = get_n_connected(idx, cartesian_positions, edge_indexes, edge_attributes)
    print()
    print(idx)
    print(n_connected_list)

    # Remove idx from connected_dict
    n_connected_list.remove(idx)
    
    # Set idx in cartesian_positions or else add it to n_connected_list
    if n_connected >= 3:
        # Extract the cartesian coordinates of idx
        temp_position = find_valid_reference(n_connected, idx_connected, edge_indexes, edge_attributes)

        # Check if there are enough particles able to locate idx (images make this step more difficult
        if temp_position is not None:
            # Generate temporal dictionary with the cartesian coordinates
            temp_dict = {
                idx: temp_position
            }

            # Update general dictionary with cartesian coordinates
            cartesian_positions.update(temp_dict)
        else:
            # n_connected_list is updated adding idx where it belongs to
            n_connected_list.append(idx)
    else:
        # n_connected_list is updated adding idx where it belongs to
        n_connected_list.append(idx)


1
[1, 3, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70]

3
[3, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 1]

5
[5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 1, 3]

6
[6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 5


64
[64, 66, 67, 68, 69, 70, 1, 3, 5, 7, 9, 11, 12, 14, 15, 16, 17, 18, 19, 21, 22, 23, 25, 27, 29, 30, 31, 32, 33, 34, 35, 37, 38, 39, 48, 50, 52, 56, 58, 63]

66
[66, 67, 68, 69, 70, 1, 3, 5, 7, 9, 11, 12, 14, 15, 16, 17, 18, 19, 21, 22, 23, 25, 27, 29, 30, 31, 32, 33, 34, 35, 37, 38, 39, 48, 50, 52, 56, 58, 63]

67
[67, 68, 69, 70, 1, 3, 5, 7, 9, 11, 12, 14, 15, 16, 17, 18, 19, 21, 22, 23, 25, 27, 29, 30, 31, 32, 33, 34, 35, 37, 38, 39, 48, 50, 52, 56, 58, 63, 66]

68
[68, 69, 70, 1, 3, 5, 7, 9, 11, 12, 14, 15, 16, 17, 18, 19, 21, 22, 23, 25, 27, 29, 30, 31, 32, 33, 34, 35, 37, 38, 39, 48, 50, 52, 56, 58, 63, 66]

69
[69, 70, 1, 3, 5, 7, 9, 11, 12, 14, 15, 16, 17, 18, 19, 21, 22, 23, 25, 27, 29, 30, 31, 32, 33, 34, 35, 37, 38, 39, 48, 50, 52, 56, 58, 63, 66]

70
[70, 1, 3, 5, 7, 9, 11, 12, 14, 15, 16, 17, 18, 19, 21, 22, 23, 25, 27, 29, 30, 31, 32, 33, 34, 35, 37, 38, 39, 48, 50, 52, 56, 58, 63, 66]

1
[1, 3, 5, 7, 9, 11, 12, 14, 15, 16, 17, 18, 19, 21, 22, 23, 25, 27, 29, 30, 31, 3

In [9]:
for i in range(total_particles):
    print(cartesian_positions[i])

[0 0 0]
[ 6.33492156 -2.15680363  2.75753612]
[4.1835289 0.        0.       ]
[ 4.43208152 -1.65717008 -0.95248837]
[2.09514309 3.65975825 0.        ]
[ 8.37187501 -0.27085107 -0.41596758]
[1.86472653 1.9855326  2.77872563]
[ 1.4867463   0.73758424 -2.02112559]
[5.86140262 3.2097081  1.24752291]
[ 5.43006632  2.11200041 -1.48936774]
[-2.02442593  1.87612993  2.81597641]
[5.59322627 1.55766173 1.70239072]
[3.6765786  0.33254059 2.90697091]
[3.47120125 2.22670121 2.74683594]
[ 5.48381763 -3.43079085  3.35439857]
[ 4.98712563 -1.69176191  2.66161519]
[ 2.96526985 -0.06015047  2.98956556]
[3.25692488 0.84581598 0.62014272]
[ 2.17418618 -1.45647761  2.33920149]
[ 4.10660636 -0.50031558  0.85119244]
[ 2.62489018 -0.53392076  5.60147924]
[ 3.21302977 -0.18439839  8.15454571]
[6.61767447 2.87317859 0.47547129]
[2.02660489 2.90035016 5.39785423]
[2.06915974 1.75913118 0.21751348]
[ 3.7292934  -2.09141017  0.42690069]
[ 2.97316479 -2.45449662  0.41113898]
[ 6.33012902 -0.16033689 -2.2579573 ]
[-